# Sarcasm Detection using Hierarchical Bert

## 1. DATA LOADING AND PREPROCESSING

In [8]:
pip install transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 989.2 kB/s eta 0:00:10
   --- ------------------------------------ 0.8/10.0 MB 987.4 kB/s eta 0:00:10
   --- ------------------------------------ 0.8/10.0 MB 987.4 kB/s eta 0:00:10
   ---- ----------------------------------- 1.0/10.0 MB 825.2 kB/s eta 0:00:11
   ----- ---------------------------------- 1.3/10.0 MB 919.8 kB/s eta 0:00:10
   ----- ---------------------------------- 1.3/10.0 MB 919.8 kB/s eta 0:00:10
   ------ --------------------------------- 1.6/10.0 MB 922.4 kB/s eta 0:00:10
   ------- -------------------------------- 1.8/10.0 MB 898.8 kB/s eta 0:00:10
   -------- ------------------------------- 2.1/10.0 MB 939.6 kB/s eta 0:00:09
   -------- ------------------------------- 2.1/10.0 MB 939.6 kB/s eta 0:00:09
   --------- ------------------------------ 2.4/10.0 MB 889.0 kB/s eta

In [10]:
pip install tf-keras

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 3.0 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.7 MB 3.0 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/390.3 MB 1.5 MB/s eta 0:04:16
   ---------------------------------------- 0.8/390.3 MB 1.8 MB/s eta 0:03:41
   ---------------------------------------- 1.0/390.3 MB 1.7 MB/s eta 0:03:45
   ------------

  You can safely remove it manually.


In [1]:
#importing libraries
import zipfile
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('train-balanced-sarcasm.csv')
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [ ]:
df.shape

(1010826, 10)

In [ ]:
df = df[:25000]
df = df[['label', 'comment']]
df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [ ]:
df.shape

(10000, 2)

In [ ]:
df.isna().sum()

,0
label,0
comment,1


In [ ]:
df.dropna(inplace = True)
df.isna().sum()

,0
label,0
comment,0


In [ ]:
#removing unwanted numerals and symbols
df['comment'] = df['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [ ]:
#converting the data into lowercase
def lowercase(text):
  return text.lower()

df['comment'] = df['comment'].apply(lowercase)

In [ ]:
df.head()

,label,comment
0,0,nc and nh
1,0,you do know west teams play against west teams...
2,0,they were underdogs earlier today but since gr...
3,0,this meme isnt funny none of the new york nigg...
4,0,i could use one of those tools


## 2. TOKENIZATION

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#function for tokenization
def tokenize_data(text, max_length = 100):
  return tokenizer(
      text.tolist(),
      max_length = max_length,
      truncation = True,
      padding = 'max_length',
      return_tensors = 'np'
  )

tokenized_data = tokenize_data(df['comment'])

In [ ]:
tokenized_data

{'input_ids': array([[  101, 13316,  1998, ...,     0,     0,     0],
       [  101,  2017,  2079, ...,     0,     0,     0],
       [  101,  2027,  2020, ...,     0,     0,     0],
       ...,
       [  101,  5095,  2305, ...,     0,     0,     0],
       [  101, 29420,  2015, ...,     0,     0,     0],
       [  101,  2016, 28719, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

## 3. TRAIN_TEST_SPLIT

In [ ]:
X = tokenized_data['input_ids']
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train.shape, X_test.shape

((7999, 100), (2000, 100))

## 4.Building the model according to the proposed architecture

In [ ]:
class HierarchicalBERT(tf.keras.Model):
    def __init__(self, bert_model, lstm_units, cnn_filters, dense_units):
        super(HierarchicalBERT, self).__init__()
        self.bert = bert_model

        # Sentence Encoding Layer
        self.dense_sentence = tf.keras.layers.Dense(768, activation='relu')

        # Context Summarization Layer
        self.mean_pooling = tf.keras.layers.GlobalAveragePooling1D()

        # Context Encoder Layer
        self.bilstm_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))

        # CNN Layer
        self.conv = tf.keras.layers.Conv1D(cnn_filters, 2, activation='relu')
        self.pool = tf.keras.layers.GlobalMaxPooling1D()

        # Fully Connected Layer
        self.dense_fc = tf.keras.layers.Dense(dense_units, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # BERT Embeddings
        bert_output = self.bert(inputs)[0]  # (batch_size, seq_len, hidden_size)

        # Sentence Encoding Layer
        sentence_encoded = self.dense_sentence(bert_output)  # (batch_size, seq_len, 768)

        # Context Summarization Layer
        context_summarized = self.mean_pooling(sentence_encoded)  # (batch_size, 768)

        # Expand dimensions to match the input shape required by LSTM
        context_summarized = tf.expand_dims(context_summarized, 1)  # (batch_size, 1, 768)

        # Context Encoder Layer
        context_encoded = self.bilstm_encoder(context_summarized)  # (batch_size, 1, 2 * lstm_units)

        # Squeeze the second dimension to match the input shape required by Conv1D
        context_encoded_squeezed = tf.squeeze(context_encoded, axis=1)  # (batch_size, 2 * lstm_units)

        # Add the channels dimension to match the input shape required by Conv1D
        context_encoded_expanded = tf.expand_dims(context_encoded_squeezed, axis=-1)  # (batch_size, 2 * lstm_units, 1)

        # CNN Layer
        conv_output = self.conv(context_encoded_expanded)  # (batch_size, new_seq_len, cnn_filters)
        pooled_output = self.pool(conv_output)  # (batch_size, cnn_filters)

        # Fully Connected Layer
        dense_output = self.dense_fc(pooled_output)  # (batch_size, dense_units)

        # Output Layer
        final_output = self.output_layer(dense_output)  # (batch_size, 1)
        return final_output


In [ ]:
#loading the pretrained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
#defining the hierarchical bert model
model = HierarchicalBERT(bert_model, lstm_units = 128, cnn_filters = 64, dense_units = 32)

In [ ]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5


250/250 [==============================] - 261s 634ms/step - loss: 0.6617 - accuracy: 0.6268
Epoch 2/5
250/250 [==============================] - 161s 644ms/step - loss: 0.6618 - accuracy: 0.6268
Epoch 3/5
250/250 [==============================] - 162s 646ms/step - loss: 0.6617 - accuracy: 0.6268
Epoch 4/5
250/250 [==============================] - 161s 644ms/step - loss: 0.6617 - accuracy: 0.6268
Epoch 5/5
250/250 [==============================] - 161s 643ms/step - loss: 0.6616 - accuracy: 0.6268


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy : {accuracy * 100}')

63/63 [==============================] - 28s 211ms/step - loss: 0.6553 - accuracy: 0.6375
Model Accuracy : 63.749998807907104
